<a href="https://colab.research.google.com/github/Nahom32/DNN-Simulation/blob/main/DNN_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Deep Neural Networks
This notebook contains an illustration of how the deep neural networks work. It implements how both forward and backward propagation works using the gradient descent algorithm.

The forward propagation algorithm works as follows:
$$Z^l = W^la^{l-1} + b^l$$
$$a^l = \alpha(Z^l)$$ where $\alpha$ is the activation function
The backward propagation algorithms uses gradient descent to autocorrect the weights. The algorihtm works as follows:
$$W^l = W^l - \frac{∂E}{∂W^l}$$

Since the Error equation isn't described interms of the internal weights, we need to use the chain rule to find the partial derivative.





## Imports


In [1]:
import numpy as np
import pandas as pd


In [ ]:
def relu(x):
  '''
  Description
  ---------------------------
  This is the relu activation function which is mostly used in hidden layers
  ===========================
  args:
    x: flloat, np.matrix
  returns:
    np.matrix
  '''
  return np.maximum(x,0)

def softmax(x):
    x_val = np.exp(x - np.max(x, axis=1, keepdims=True))
    return x_val / np.sum(x_val, axis=1, keepdims=True)

def sigmoid(x):
  '''
  Description
  ---------------------------
  This is the sigmoid activation function
  ===========================
  args:
    x: flloat, np.matrix
  returns:
    np.matrix
  '''
  return 1/(1+np.exp(-x))


